# Projet 3 : Concevez une application au service de la santé publique

## Partie 1 : nettoyage du jeux de données

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

write_data = False

In [2]:
# si besoin importer le fichier
# import os.path
# import wget
# if os.path.exists('en.openfoodfacts.org.products.csv'):
#    print("Le jeu de données est déjà présent dans le répertoire")
# else:  
#    wget.download("https://static.openfoodfacts.org/data/en.openfoodfacts.org.products.csv", out='.')

### Ouverture du fichier .csv

In [3]:
# ouverture du fichire csv
# utilise le fichier dans le répertoire si il existe 
# sinon récupération avec l'url
if os.path.exists('en.openfoodfacts.org.products.csv'):
    OpenFoodData = pd.read_csv(
        'en.openfoodfacts.org.products.csv',
        sep='\t',  # séparateurs = tabulations
        low_memory=False)
else:
    OpenFoodData = pd.read_csv(
        "https://static.openfoodfacts.org/data/en.openfoodfacts.org.products.csv",
        sep='\t',  # séparateurs = tabulations
        low_memory=False)


Afin de réduire l'usage de la mémoire de l'ordinateur nous allons changer le type de certaines colonnes

In [4]:
# on retire les colonnes de NaN
OpenFoodData.dropna(axis=1, how='all', inplace=True)


In [5]:
# on réduit la taille des colonnes contenant des entiers si possible
for x in OpenFoodData.select_dtypes(include=[int]).columns.to_list():
    if (OpenFoodData[x].max() <= 127) & (OpenFoodData[x].min() >= -128):
        OpenFoodData[x] = OpenFoodData[x].astype('int8')
    elif (OpenFoodData[x].max() <= 32767) & (OpenFoodData[x].min() >= -32768):
        OpenFoodData[x] = OpenFoodData[x].astype('int16')
    elif (OpenFoodData[x].max() <= 2147483647) & (OpenFoodData[x].min() >=
                                                  -2147483648):
        OpenFoodData[x] = OpenFoodData[x].astype('int32')
    else:
        OpenFoodData[x] = OpenFoodData[x]


In [6]:
# on réduit la taille des colonnes contenant des nombres décimaux
OpenFoodDataNFloat = OpenFoodData.select_dtypes(exclude=[float])
OpenFoodDataFloat = OpenFoodData.select_dtypes(include=[float]).astype('float32')
OpenFoodData = OpenFoodDataNFloat.join(OpenFoodDataFloat)

In [7]:
# vérification des suppressions duent à la réduction 
# de la taille de nombres décimaux
inf = OpenFoodDataFloat.where(OpenFoodDataFloat == np.inf).dropna(axis=0,
                                                                  how='all')
inf = inf.where(inf == np.inf).dropna(axis=1, how='all')
print(inf)

        energy-kj_100g  energy_100g
658122             inf          inf


Une seule valeur d'énergie est perdue en réduisant la taille des nombres décimaux. On peut considérer ça comme négligeable au vu de la taille du fichier d'autant que c'était sûrement une valeur aberrante pour être aussi importante

In [8]:
columns_energy = OpenFoodData.loc[:, OpenFoodData.columns.str.contains('energy')]
columns_Nenergy = OpenFoodData.loc[:, ~OpenFoodData.columns.str.contains('energy')]
columns_energy.replace(np.inf, np.nan, inplace=True)
OpenFoodData = columns_Nenergy.join(columns_energy)


/home/lancelot/PyDS/lib/python3.9/site-packages/pandas/core/frame.py:5238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [9]:
# 3 premières lignes du fichier
OpenFoodData.head(3)

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,abbreviated_product_name,generic_name,...,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g,energy-kj_100g,energy-kcal_100g,energy_100g,energy-from-fat_100g
0,00000000000000225,http://world-en.openfoodfacts.org/product/0000...,nutrinet-sante,1623855208,2021-06-16T14:53:28Z,1623855209,2021-06-16T14:53:29Z,jeunes pousses,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00000000000003429145,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1630483911,2021-09-01T08:11:51Z,1630484064,2021-09-01T08:14:24Z,L.casei,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0000000000017,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1529059080,2018-06-15T10:38:00Z,1561463718,2019-06-25T11:55:18Z,Vitória crackers,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,375.0,1569.0,NaN


Entêtes des colonnes

In [10]:
# liste des entêtes de colonnes
OpenFoodData.columns.to_list()

['code',
 'url',
 'creator',
 'created_t',
 'created_datetime',
 'last_modified_t',
 'last_modified_datetime',
 'product_name',
 'abbreviated_product_name',
 'generic_name',
 'quantity',
 'packaging',
 'packaging_tags',
 'packaging_text',
 'brands',
 'brands_tags',
 'categories',
 'categories_tags',
 'categories_en',
 'origins',
 'origins_tags',
 'origins_en',
 'manufacturing_places',
 'manufacturing_places_tags',
 'labels',
 'labels_tags',
 'labels_en',
 'emb_codes',
 'emb_codes_tags',
 'first_packaging_code_geo',
 'cities_tags',
 'purchase_places',
 'stores',
 'countries',
 'countries_tags',
 'countries_en',
 'ingredients_text',
 'allergens',
 'traces',
 'traces_tags',
 'traces_en',
 'serving_size',
 'additives_tags',
 'additives_en',
 'ingredients_from_palm_oil_tags',
 'ingredients_that_may_be_from_palm_oil_tags',
 'nutriscore_grade',
 'pnns_groups_1',
 'pnns_groups_2',
 'states',
 'states_tags',
 'states_en',
 'brand_owner',
 'ecoscore_grade_fr',
 'main_category',
 'main_category_e

Il y a plusieurs types de colonnes contenant des dates : 'created_t', 'created_datetime', 'last_modified_t', 'last_modified_datetime'. Les colonnes finissant par "_t" sont en timestamp c'est à dire le nombre de secondes écoulées debuis le 1er janvier 1970 (début de l'heure UNIX). Les colonnes finissant par "_datetime" sont au format : yyyy-mm-ddThh:mn:ssZ.

Nous allons supprimer les timestamps et mettre au format date les colonnes datetime

In [11]:
OpenFoodData.loc[:, OpenFoodData.columns.str.endswith('_t')].head(3)

,created_t,last_modified_t
0,1623855208,1623855209
1,1630483911,1630484064
2,1529059080,1561463718


In [12]:
OpenFoodData.loc[:, OpenFoodData.columns.str.endswith('_datetime')].head(3)

,created_datetime,last_modified_datetime
0,2021-06-16T14:53:28Z,2021-06-16T14:53:29Z
1,2021-09-01T08:11:51Z,2021-09-01T08:14:24Z
2,2018-06-15T10:38:00Z,2019-06-25T11:55:18Z


In [13]:
# suppression timestamp
OFDClean = OpenFoodData.loc[:, ~OpenFoodData.columns.str.endswith('_t')]
# mise au format temps
OFDClean.loc[:, OFDClean.columns.str.endswith('_datetime')].apply(
    lambda x: pd.to_datetime(x, infer_datetime_format=True))


,created_datetime,last_modified_datetime
0,2021-06-16 14:53:28,2021-06-16 14:53:29
1,2021-09-01 08:11:51,2021-09-01 08:14:24
2,2018-06-15 10:38:00,2019-06-25 11:55:18
3,2018-10-13 21:06:14,2018-10-13 21:06:57
4,2019-11-19 15:02:16,2021-06-22 19:39:25
...,...,...
1991901,2019-10-31 09:24:26,2019-10-31 09:24:26
1991902,2020-12-16 07:58:23,2020-12-16 07:58:24
1991903,2020-02-08 14:20:13,2020-02-25 15:24:07
1991904,2021-02-12 11:35:28,2021-02-12 11:35:30


Étant donné qu'on travail pour Santé publique France on va se concentrer sur les prroduits vendus en France

In [14]:
# sélection des produits vendu en france
OFDCleanFR = OFDClean[OFDClean.countries.str.contains('fr',
                                                      case=False,
                                                      na=False)]


In [15]:
px.bar(x=OFDCleanFR.shape[0] - OFDCleanFR.isna().sum().sort_values(ascending=False).values,
       y=OFDCleanFR.isna().sum().sort_values(ascending=False).index,
       labels=dict(x='Nombre de données', y='Indicateurs'),
       height=3000)


In [16]:
# filtrer les colonnes concernant la quantité pour 100g entre 0 et 100 pour éliminer les valeurs aberrantes
# fonction de selection des colonnes en fonction du nom
columns_100g_small = OFDCleanFR.loc[:, (
    OFDCleanFR.columns.str.endswith('_100g')
    & ~OFDCleanFR.columns.str.contains('energy|footprint|nutrition|water'))]
OFDFilt = columns_100g_small.mask(
    ((columns_100g_small < 0) | (columns_100g_small > 100)), other=np.nan)
OFDFiltFull = OFDCleanFR.loc[:,
                             ~OFDCleanFR.columns.isin(OFDFilt.columns)].join(
                                 OFDFilt)


In [17]:
# suppression des données avec plus de 99% de NaN
OFDFiltFull.dropna(axis='columns',
                   thresh=(OFDFiltFull.shape[0] * .01),
                   inplace=True)


In [18]:
px.bar(x=OFDFiltFull.shape[0] -
       OFDFiltFull.isna().sum().sort_values(ascending=False).values,
       y=OFDFiltFull.isna().sum().sort_values(ascending=False).index,
       labels=dict(x='Nombre de données', y='Indicateurs'),
       height=1500)

In [19]:
# sélection des colonnes qui finissent par _100g
columns_100g = OFDFiltFull.loc[:, (OFDFiltFull.columns.str.endswith('_100g'))]


In [20]:
columns_num = OFDFiltFull.select_dtypes('number')

### Visualisation des distributions dans les colonnes numériques

In [21]:
def dist_subplot(df, title):
    if len(df.columns) == 2:
        nrows=1
    elif len(df.columns) == 5:
        nrows=2
    else:
        nrows = int(np.ceil(np.sqrt(len(df.columns))))
    ncols = int(np.ceil(np.sqrt(len(df.columns))))
    fig = make_subplots(rows=nrows,
                        cols=ncols,
                        subplot_titles=df.columns,
                        x_title='value',
                        y_title='count')
    col = 1
    row = 1
    for c in df.columns:
        fig.add_trace(go.Histogram(x=df[c],
                                   name=df[c].name,
                                   nbinsx=100,
                                   showlegend=False,
                                   marker=dict(color='blue')),
                      row=row,
                      col=col)
        if col < ncols:
            row = row
            col += 1
        else:
            row += 1
            col = 1
    fig.update_layout(height=nrows * 200, width=ncols * 300, title=title)
    return (fig.show(renderer='browser'))


dist_subplot(columns_num,'Distribution dans les colonnes numériques')

Il doit il y avoir des valeurs extrèmes pour la quantité servie, l'empreinte carbone et l'energie car on observe qu'une seule barre et des valeurs très importantes où le nombre de valeurs est négligeable. Nous allons donc écarter ces valeurs lorsqu'elles sont en dehors du quartile qui contient 99% des valeurs. 

### Nettoyage des outliers

In [22]:
# calcul du quartile dans lequel 99% des valeurs sont contenus
q3 = OFDFiltFull.select_dtypes(
    'number').loc[:,
                  OFDFiltFull.select_dtypes('number').columns.str.
                  contains('energy|serving|carbon')].quantile(.99)
q3.index

Index(['serving_quantity', 'carbon-footprint-from-meat-or-fish_100g',
       'energy-kj_100g', 'energy-kcal_100g', 'energy_100g'],
      dtype='object')

In [23]:
# filtre des valeurs au dessus du quartile .99
for x in q3.index:
    OFDFiltFull.loc[:, (OFDFiltFull.columns == x)] = OFDFiltFull.loc[:, (
        OFDFiltFull.columns == x)].where(
            OFDFiltFull.loc[:, (OFDFiltFull.columns == x)] < q3[x])

dist_subplot(
    OFDFiltFull.select_dtypes(
        'number').loc[:,
                      OFDFiltFull.select_dtypes('number').columns.str.
                      contains('energy|serving|carbon')], 'Distribution dans les colonnes quantité, empreinte carbone et energies')


In [24]:
if write_data is True:
    OFDFiltFull.to_csv('FR.openfoodfacts.org.products.csv', index=False)